In [1]:
!mamba install altair
!mamba install pandas
!mamba install matplotlib

mambajs 0.19.13

Specs: xeus-python, numpy, matplotlib, pillow, ipywidgets>=8.1.6, ipyleaflet, scipy, altair
Channels: emscripten-forge, conda-forge

Solving environment...
Solving took 1.688600000023842 seconds
  Name                          Version                       Build                         Channel                       
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
+ altair                        6.0.0                         pyhd8ed1ab_1                  conda-forge                   
+ attrs                         25.4.0                        pyhcf101f3_1                  conda-forge                   
+ importlib_resources           6.5.2                         pyhd8ed1ab_0                  conda-forge                   
+ importlib-metadata            8.7.0                         pyhe01879c_1                  conda-forge                   
+ jsonschema                    4.17.3              

In [2]:
# put all imports here

import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
import re

In [22]:
# Cleaning

washis_df_fin = pd.read_csv("WAHIS15-19.csv")

washis_df_fin.head()

,Year,Semester,World region,Country,Administrative Division,Disease,Serotype/Subtype/Genotype,Animal Category,Event_id,Species,Outbreak_id,New outbreaks,Susceptible,Measuring units,Cases,Killed and disposed of,Slaughtered,Deaths,Vaccinated
0,2015,Jan-Jun 2015,Africa,Algeria,Algeria,Bovine tuberculosis (-2018),NaN,Both animal categories,-,NaN,-,45,-,-,-,-,-,-,-
1,2015,Jan-Jun 2015,Africa,Algeria,Algeria,Bovine tuberculosis (-2018),NaN,Domestic,-,Cattle,-,-,901,Animal,211,0,211,0,-
2,2015,Jan-Jun 2015,Africa,Algeria,Algeria,Brucella abortus (Inf. with),NaN,Both animal categories,-,NaN,-,293,-,-,-,-,-,-,-
3,2015,Jan-Jun 2015,Africa,Algeria,Algeria,Brucella abortus (Inf. with),NaN,Domestic,-,Cattle,-,-,2945,Animal,657,0,657,0,0
4,2015,Jan-Jun 2015,Africa,Algeria,Algeria,Brucella melitensis (Inf. with),NaN,Both animal categories,-,NaN,-,32,-,-,-,-,-,-,-


In [23]:
# List of known zoonotic diseases (verified)
zoonDiseaseList = [
    'Anthrax',
    'Avian chlamydiosis',
    'Avian influenza A viruses of high pathogenicity (Inf. with)',
    'Bovine babesiosis',
    'Bovine spongiform encephalopathy (BSE)',
    'Bovine tuberculosis',
    'Brucella abortus',
    'Brucella melitensis',
    'Brucella suis',
    'Burkholderia mallei (Glanders)',
    'Camelpox virus (rare zoonosis)',
    'Chlamydia abortus',
    'Chrysomya bezziana (Old World screwworm)',
    'Cochliomyia hominivorax (New World screwworm)',
    'Coxiella burnetii (Q fever)',
    'Crimean–Congo haemorrhagic fever',
    'Eastern equine encephalitis virus',
    'Echinococcus granulosus',
    'Echinococcus multilocularis',
    'Foot and mouth disease virus (rare zoonosis)',
    'Japanese encephalitis virus',
    'Leishmania spp.',
    'Low pathogenicity avian influenza viruses transmissible to humans',
    'Middle East respiratory syndrome coronavirus (MERS-CoV)',
    'Mycobacterium tuberculosis complex',
    'Newcastle disease virus',
    'Pox viruses (other than those listed by the OIE)',
    'Rabies virus',
    'Rift Valley fever virus',
    'SARS-CoV-2 in animals',
    'Taenia solium',
    'Trichinella spp.',
    'Trypanosoma brucei',
    'Tularemia',
    'Venezuelan equine encephalomyelitis',
    'West Nile Fever',
    'Western equine encephalitis virus'
]


In [24]:
# Start from raw WAHIS zoonotic data
washis_df = washis_df_fin.copy()

# Normalize disease names
washis_df["Disease_norm"] = (
    washis_df["Disease"]
    .astype(str)
    .str.strip()
    .str.lower()
)

zoon_terms_sorted = sorted(
    {d.lower().strip() for d in zoonDiseaseList}
)

# Keep zoonotic diseases only
washis_df = washis_df[
    washis_df["Disease_norm"].isin(zoon_terms_sorted)
]

# Numeric cases
washis_df["Cases_num"] = pd.to_numeric(
    washis_df["Cases"], errors="coerce"
).fillna(0)

# Aggregate to Country–Year
ZoonCasesbyCountryYear = (
    washis_df
    .groupby(["Country", "Year"], as_index=False)["Cases_num"]
    .sum()
    .rename(columns={"Cases_num": "Zoonotic_Cases"})
)


In [25]:
# Create scaffold to clean up country and year columns

country_year_scaffold = (
    washis_df
    [["Country", "Year"]]
    .drop_duplicates()
    .assign(
        Country=lambda d: d["Country"].astype(str).str.strip(),
        Year=lambda d: d["Year"].astype(int)
    )
)
ZoonCasesbyCountryYear_final = (
    country_year_scaffold
    .merge(
        ZoonCasesbyCountryYear,
        how="left",
        on=["Country", "Year"]
    )
    .fillna({"Zoonotic_Cases": 0})
    .sort_values(["Country", "Year"])
    .reset_index(drop=True)
)

# Optional index style to match screenshot
ZoonCasesbyCountryYear_final.index = range(1, len(ZoonCasesbyCountryYear_final) + 1)


In [26]:
ZoonCasesbyCountryYear_final


,Country,Year,Zoonotic_Cases
1,Afghanistan,2015,3644.0
2,Afghanistan,2016,1566.0
3,Afghanistan,2017,3008.0
4,Afghanistan,2018,3633.0
5,Afghanistan,2019,3320.0
...,...,...,...
480,Zimbabwe,2015,1639.0
481,Zimbabwe,2016,1476.0
482,Zimbabwe,2017,1699.0
483,Zimbabwe,2018,1706.0


In [38]:
# this dataset primarily is used to analyze the production of crops and livestock (density)
# Loading the dataset from csv

Prod_df_fin = pd.read_csv("Production_Crops_Livestock_E_All_Data_NOFLAG.csv")

# Drop unnecessary years
SlimTradeDF = Prod_df_fin.drop(columns=['Area Code', 'Area Code (M49)', 'Item Code', 
       'Item Code (CPC)', 'Element Code', 'Y1961', 'Y1962', 'Y1963',
       'Y1964', 'Y1965', 'Y1966', 'Y1967', 'Y1968', 'Y1969', 'Y1970', 'Y1971',
       'Y1972', 'Y1973', 'Y1974', 'Y1975', 'Y1976', 'Y1977', 'Y1978', 'Y1979',
       'Y1980', 'Y1981', 'Y1982', 'Y1983', 'Y1984', 'Y1985', 'Y1986', 'Y1987',
       'Y1988', 'Y1989', 'Y1990', 'Y1991', 'Y1992', 'Y1993', 'Y1994', 'Y1995',
       'Y1996', 'Y1997', 'Y1998', 'Y1999', 'Y2000', 'Y2001', 'Y2002', 'Y2003',
       'Y2004', 'Y2005', 'Y2006', 'Y2007', 'Y2008', 'Y2009', 'Y2010', 'Y2011',
       'Y2012', 'Y2013', 'Y2014', 'Y2020', 'Y2021', 'Y2022', 'Y2023', 'Y2024'])

ExportOnlySlimTradeDF = SlimTradeDF[SlimTradeDF["Element"] == 'Production']


# Reshaping SlimTradeDF so it can be joined with washis_df_fin. 
# Take values in each year column and add it as a row. 
trade_df_fin = ExportOnlySlimTradeDF.melt(
    id_vars=[
        "Area",
        "Item",
        "Element",
        "Unit"
    ],
    value_vars=["Y2015", "Y2016", "Y2017", "Y2018", "Y2019"],
    var_name="Year",
    value_name="Trade_Value"
)
trade_df_fin["Year"] = trade_df_fin["Year"].str.replace("Y", "").astype(int)

'''for area in trade_df_fin["Area"].dropna().unique():
    print(area)'''


'for area in trade_df_fin["Area"].dropna().unique():\n    print(area)'

In [39]:
COUNTRY_CANONICAL_MAP = {
    # Russia / Turkey (easy)
    "Russia": "Russian Federation",
    "Türkiye": "Turkey",
    "Türkiye (Rep. of)": "Turkey",

    # China handling
    "China, mainland": "China",
    "China (People's Rep. of)": "China",  
    "China, Hong Kong SAR": "China",
    "China, Macao SAR": "China",
    # Explicitly DO NOT merge these into China
    "China, Taiwan Province of": None,
}


In [40]:
trade_df_fin = trade_df_fin.copy()

trade_df_fin["Country_std"] = (
    trade_df_fin["Area"]
    .astype(str)
    .str.strip()
    .replace(COUNTRY_CANONICAL_MAP)
)


In [43]:
trade_df_fin = trade_df_fin.rename(columns={"Area": "Country"})

merged_df2 = trade_df_fin.merge(
    ZoonCasesbyCountryYear_final,
    how="left",
    left_on=["Country", "Year"],
    right_on=["Country", "Year"]
)
region_map = (
    washis_df
    .dropna(subset=["World region"])
    .assign(
        Country=lambda d: d["Country"].astype(str).str.strip(),
        Year=lambda d: d["Year"].astype(int)
    )
    .drop_duplicates(subset=["Country", "Year"])
    [["Country", "Year", "World region"]]
)


merged_df2 = (
    trade_df_fin
    .assign(
        Country=lambda d: d["Country"].astype(str).str.strip(),
        Year=lambda d: d["Year"].astype(int)
    )
    .merge(
        ZoonCasesbyCountryYear,
        how="left",
        left_on=["Country", "Year"],
        right_on=["Country", "Year"]
    )
)

merged_df2 = merged_df2.merge(
    region_map,
    how="left",
    left_on=["Country", "Year"],
    right_on=["Country", "Year"],
    suffixes=("", "_region")
)


merged_df2.head()

,Country,Item,Element,Unit,Year,Trade_Value,Country_std,Zoonotic_Cases,World region
0,Afghanistan,"Almonds, in shell",Production,t,2015,24246.0,Afghanistan,3644.0,Asia
1,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",Production,t,2015,18000.0,Afghanistan,3644.0,Asia
2,Afghanistan,Apples,Production,t,2015,89733.0,Afghanistan,3644.0,Asia
3,Afghanistan,Apricots,Production,t,2015,87686.0,Afghanistan,3644.0,Asia
4,Afghanistan,Barley,Production,t,2015,403000.0,Afghanistan,3644.0,Asia


In [48]:
# Clean df to only include one instance of country, year, world region, and case
cases_base = (
    merged_df2[
        ["Country", "Year", "World region", "Zoonotic_Cases"]
    ]
    .drop_duplicates(subset=["Country", "Year"])
    .copy()
)

cases_base["Zoonotic_Cases"] = pd.to_numeric(cases_base["Zoonotic_Cases"], errors="coerce").fillna(0)

# Group by year and country and count the number of outbreaks and obtain the total number of outbreaks
outbreak_by_country_year = (
    cases_base
    .groupby(["Country", "Year", "World region"], as_index=False)
    .agg(total_cases=("Zoonotic_Cases", "sum"))
)

df = outbreak_by_country_year.copy()
df["Year"] = df["Year"].astype(int)
df["Country"] = df["Country"].astype(str).str.strip()
df["key"] = df["Country"] + "_" + df["Year"].astype(str)

region_options = sorted(df["World region"].unique().tolist())
region_select = alt.selection_point(
    name="RegionSelect",
    fields=["World region"],
    bind=alt.binding_select(options=region_options, name="Region: "),
    value=[{"World region": region_options[0]}],
)

heatmap = (
    alt.Chart(df)
    .mark_rect()
    .transform_filter(region_select)
    .encode(
        x=alt.X("Year:O", title="Year"),
        y=alt.Y("Country:N", title="Country"),
        color=alt.Color(
            "total_cases:Q",
            title="Total Zoonotic Cases",
            scale=alt.Scale(scheme="tealblues"),
        ),
        tooltip=[
            alt.Tooltip("Country:N", title="Country"),
            alt.Tooltip("Year:O", title="Year"),
            alt.Tooltip("total_cases:Q", title="Total Cases", format=","),
            alt.Tooltip("World region:N", title="Region"),
        ],
    )
    .properties(
        width=800,
        height=600,
        title="Total Zoonotic Case Counts by Country and Year"
    )
    .add_params(region_select)
)

heatmap

# Note that poland had a huge outbreak in 2015
# Common west nile outbreaks occur in egypt

alt.Chart(...)

In [54]:
# Ensure Cases column is numeric; coerce non-numeric to NaN then fill 0
washis_df = washis_df_finZoon.copy()

#washis_df
# Normalize Disease for matching
washis_df['Disease_norm'] = washis_df['Disease'].astype(str).str.strip().str.lower()
zoon_norm = {d: d.lower().strip() for d in zoonDiseaseList}
zoon_terms_sorted = sorted(set(zoon_norm.values()))

# Create a boolean mask for zoonotic sdisease presence using exact match on normalized names
mask = washis_df['Disease_norm'].isin(zoon_terms_sorted)

# Prepare Cases as numeric
cases_numeric = pd.to_numeric(washis_df['Cases'], errors='coerce').fillna(0)
washis_df['Cases_num'] = cases_numeric

# Filter to zoonotic rows
zoo_df = washis_df.loc[mask].copy()

# Group by Country and Year, summing Cases_num
zoo_cases_by_country_year = (
    zoo_df.groupby(['Country', 'Year'], as_index=False)['Cases_num'].sum()
    .rename(columns={'Cases_num': 'Zoonotic_Cases'})
)

# Set index starting at 1
ZoonCasesbyCountryYear = zoo_cases_by_country_year.copy()
ZoonCasesbyCountryYear.index = pd.RangeIndex(start=1, stop=len(ZoonCasesbyCountryYear) + 1)

# Show the resulting dataframe
ZoonCasesbyCountryYear

<class 'NameError'>: name 'washis_df_finZoon' is not defined

In [50]:
AvgZoonCasesbyCountrySampSize = (
    ZoonCasesbyCountryYear.groupby('Country', as_index=False)
    .agg(Avg_Zoonotic_Cases=('Zoonotic_Cases', 'mean'), Sample_Size=('Year', 'nunique'))
)

# Set index starting at 1
AvgZoonCasesbyCountrySampSize.index = pd.RangeIndex(start=1, stop=len(AvgZoonCasesbyCountrySampSize) + 1)

# Create HightoLow_AvgZnCasesbyCountrySS sorted by Avg_Zoonotic_Cases descending
HightoLow_AvgZnCasesbyCountrySS = AvgZoonCasesbyCountrySampSize.sort_values(by='Avg_Zoonotic_Cases', ascending=False).copy()
HightoLow_AvgZnCasesbyCountrySS.index = pd.RangeIndex(start=1, stop=len(HightoLow_AvgZnCasesbyCountrySS) + 1)



# Preview both DataFrames
AvgZoonCasesbyCountrySampSize, HightoLow_AvgZnCasesbyCountrySS

(         Country  Avg_Zoonotic_Cases  Sample_Size
 1    Afghanistan              3034.2            5
 2        Albania                12.8            5
 3         Angola               226.0            5
 4      Argentina                63.8            5
 5        Armenia                 2.0            1
 ..           ...                 ...          ...
 125      Uruguay               343.2            5
 126    Venezuela                83.4            5
 127      Vietnam                 2.0            1
 128       Zambia               558.4            5
 129     Zimbabwe              1959.8            5
 
 [129 rows x 3 columns],
               Country  Avg_Zoonotic_Cases  Sample_Size
 1               Egypt             40782.4            5
 2            Ethiopia              7254.2            5
 3               India              5111.0            5
 4           Sri Lanka              3942.6            5
 5         Afghanistan              3034.2            5
 ..                ...   

In [51]:
dftop10HtL_AvgZnCasesbyCountry = HightoLow_AvgZnCasesbyCountrySS.head(10).drop(columns="Sample_Size")
dftop10HtL_AvgZnCasesbyCountry

,Country,Avg_Zoonotic_Cases
1,Egypt,40782.4
2,Ethiopia,7254.2
3,India,5111.0
4,Sri Lanka,3942.6
5,Afghanistan,3034.2
6,Zimbabwe,1959.8
7,Türkiye (Rep. of),1851.6
8,China (People's Rep. of),1753.0
9,Russia,1286.8
10,Burundi,1190.5


In [52]:
# Flag colors
'''flag_colors = {
    'Egypt': '#CE1126',
    'Ethiopia': '#008000',
    'India': '#FF9933',
    'Sri Lanka': '#8B5A2B',
    'Afghanistan': '#000000',
    'Zimbabwe': '#FFD700',
    'Türkiye (Rep. of)': '#C1272D',
    "China (People's Rep. of)": '#239F40',
    'Russia': '#0000FF',   # default blue
    'Burundi': '#A4C639'   # default greenish
}'''

flags = {
    'Egypt': '🇪🇬',
    'Ethiopia': '🇪🇹',
    'India': '🇮🇳',
    'Sri Lanka': '🇱🇰',
    'Afghanistan': '🇦🇫',
    'Zimbabwe': '🇿🇼',
    'Türkiye (Rep. of)': '🇹🇷',
    "China (People's Rep. of)": '🇨🇳',
    'Russia': '🇷🇺',   
    'Burundi': '🇧🇮' 
}

# Assign colors
#dftop10HtL_AvgZnCasesbyCountry['Color'] = dftop10HtL_AvgZnCasesbyCountry['Country'].map(flag_colors)

dftop10HtL_AvgZnCasesbyCountry['Country'] = dftop10HtL_AvgZnCasesbyCountry['Country'].map(
    lambda country: f"{flags[country]} {country}" if country in flags else country
)


In [53]:

base = alt.Chart(dftop10HtL_AvgZnCasesbyCountry).encode(
    x=alt.X('Avg_Zoonotic_Cases:Q', title=None,
            axis=alt.Axis(grid=False, domain=False, labels=False,
                         ticks=False)),
    y=alt.Y('Country:N',
            axis=alt.Axis(domainColor="lightgray", 
                                       grid=False,
                                       ticks=True, tickSize=13, 
                                       tickColor="white", labelColor="gray", labelFontSize=12), sort='-x', title=None))



rules = base.mark_rule(color='#5F9EA0', opacity=0.60)
    

# Circles (heads of the lollipop)
points = base.mark_circle(size=100, color='#5F9EA0', strokeWidth=0, opacity=0.85).encode(
    x='Avg_Zoonotic_Cases:Q'
)

# Text labels next to the points
text = base.mark_text(align='left', dx=15, dy=3).encode(
    x='Avg_Zoonotic_Cases:Q',
    text=alt.Text('Avg_Zoonotic_Cases:Q', format=',.0f'),
    color=alt.value("black")
)

# Final chart layout
chart = (rules + points + text).properties(
    width=600,
    height=400,
    title=alt.Title(text='Average Zoonotic Cases / Year',
                    subtitle= "Top 10 Countries From 2015-2019",
                    color="#5F9EA0", dy=-10, dx=10, subtitleColor="gray", subtitleFontSize=10.5
    
))

chart.configure_view(strokeWidth=0)


alt.LayerChart(...)